In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import Imputer,MaxAbsScaler
from sklearn.pipeline import Pipeline,FeatureUnion

#gathering data

In [ ]:
data1=pd.read_csv('test_data_week_1_challenge.csv')
data2=pd.read_csv('train_data_week_1_challenge.csv')

In [ ]:
data2.head(10)

In [ ]:
data2.tail(10)

#exploratory data analysis

In [ ]:
data1.shape

In [ ]:
data2.shape

In [ ]:
data2.hist(figsize=(16,16))

dealing with missing values

Cleaning data will checkout all the missing value. Whenever their is a missing value: replace it with 0, or with min(), with median(), or remove the whole row

In [ ]:
data2.isnull().sum()

in this case we are going to remove the rows with missing values

In [ ]:
data2.drop(['total_students','establishment_year'],axis=1,inplace=True)

In [ ]:
data2.head()

data transformation

In [ ]:
data2=data2.replace({'drop':0,'continue':1})

In [ ]:
data2.head()

In [ ]:
data2=data2.replace({'F':0,'M':1})

In [ ]:
data2

In [ ]:
data2=data2.replace({'BC':0,'OC':1,'SC':2,'ST':3})

In [ ]:
data2=data2.replace({'father':0,'mother':1,'other':2})

In [ ]:
data2=data2.replace({False:0,True:1})

In [ ]:
data2.head()

Dealing with imbalanced data

In [ ]:
data2.drop(['student_id','school_id'],axis=1,inplace=True)

In [ ]:
data2.continue_drop.value_counts()

In [ ]:
data2_many=data2[data2['continue_drop']==1]
data2_few=data2[data2['continue_drop']==0]
count_1,count_0=data2['continue_drop'].value_counts()
sample1=data2_few.sample(count_1,replace=True)
data2_balanced=pd.concat([sample1,data2_many],axis=0)
data2_balanced['continue_drop'].value_counts()

In [ ]:
data2.hist(figsize=(16,16))

In [ ]:
data2.shape

In [ ]:
data2.size

model selection


In [ ]:
data2.shape

In [ ]:
data2.describe().head(10)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features=data2_balanced.drop(['continue_drop'],axis=1)
target=data2_balanced['continue_drop'].copy()


In [ ]:
data2, data1, data2_og, data1_og = train_test_split(features,target , test_size=.25, random_state=1)

In [ ]:
class DataFrameSelection(TransformerMixin,BaseEstimator):
    def _init_(self,attributes_names):
         self.attributes_names=attributes_names
    def fit(self,x,y=0):
            return self
    def transform(self,x,y=None):
        return x[self.attributes_names].values
    

In [ ]:
numerical_attributes=data2.columns
numerical_attributes

In [ ]:
features.head()

In [ ]:
numerical_attributes=data2.columns

In [ ]:
numerical_pipeline= Pipeline([('selector',DataFrameSelection()),
                             ('scaler',MaxAbsScaler()),
                             ('imputer',Imputer(strategy='mean'))])

In [ ]:
final_pipeline = FeatureUnion(transformer_list=[('num_pipeline',numerical_pipeline)])

In [ ]:
prepared_data =final_pipeline.fit_transform(data2_og)

In [ ]:
train_df=pd.DataFrame(final_pipeline)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNN=KNeighborsClassifier()

In [ ]:
KNN.fit(train_df, data2_og)
y_pred = KNN.predict(data2)

evaluation


In [ ]:
KNN.score(data1,data1_og)

In [ ]:
from sklearn import f1_score

In [ ]:
f1_score(data1_og,y_pred)

In [ ]:
from sklearn.metrics import confusion_matrics

In [ ]:
conf_matrix=confusion_matrics(data1,y_pred)

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(conf_matrix,annot=True,cmap='Blue',cbar='False')

save predictions

In [ ]:
np.save("Predict.npy",y_pred)